In [1]:
import pandas as pd
from matplotlib import pyplot as plt
from matplotlib import warnings

In [11]:
dataAssets = pd.read_csv('my_stock_assets.csv');

symbols = dataAssets['Symbol']

In [12]:
from datetime import datetime
import backtrader as bt

# Create a subclass of Strategy to define the indicators and logic

class SmaCross1030(bt.Strategy):
    # list of parameters which are configurable for the strategy
    params = dict(
        pfast=10,  # period for the fast moving average
        pslow=30   # period for the slow moving average
    )
    
    def log(self, txt, dt=None):
        dt = dt or self.datas[0].datetime.date(0)
        print(f'{dt.isoformat()} {txt}') #Print date and close

    def __init__(self):
        sma1 = bt.ind.SMA(period=self.p.pfast)  # fast moving average
        sma2 = bt.ind.SMA(period=self.p.pslow)  # slow moving average
        self.crossover = bt.ind.CrossOver(sma1, sma2)  # crossover signal

    def next(self):
        if not self.position:  # not in the market
            if self.crossover > 0:  # if fast crosses slow to the upside
                self.buy()  # enter long
                print(f'{self.data.p.dataname}')
                self.log('Sinal de compra')
                
                

        elif self.crossover < 0:  # in the market & cross to the downside
            self.close()  # close long position
            self.log('Sinal de venda')
    
class SmaCross1020(bt.Strategy):
    # list of parameters which are configurable for the strategy
    params = dict(
        pfast=10,  # period for the fast moving average
        pslow=20   # period for the slow moving average
    )
    
    def log(self, txt, dt=None):
        dt = dt or self.datas[0].datetime.date(0)
        print(f'{dt.isoformat()} {txt}') #Print date and close

    def __init__(self):
        sma1 = bt.ind.SMA(period=self.p.pfast)  # fast moving average
        sma2 = bt.ind.SMA(period=self.p.pslow)  # slow moving average
        self.crossover = bt.ind.CrossOver(sma1, sma2)  # crossover signal

    def next(self):
        if not self.position:  # not in the market
            if self.crossover > 0:  # if fast crosses slow to the upside
                self.buy()  # enter long
                print(f'{self.data.p.dataname}')
                self.log('Sinal de compra')
                

        elif self.crossover < 0:  # in the market & cross to the downside
            self.close()  # close long position
            self.log('Sinal de venda')
            print(f'{self.data.p.dataname}')
            

In [13]:
class TestStrategy(bt.Strategy):
    params = (('BBandsperiod', 21),)

    def log(self, txt, dt=None):
        ''' Logging function fot this strategy'''
        dt = dt or self.datas[0].datetime.date(0)
        print('%s, %s' % (dt.isoformat(), txt))

    def __init__(self):
        print(f'{self.data.p.dataname}')
        # Keep a reference to the "close" line in the data[0] dataseries
        self.dataclose = self.datas[0].close

        # To keep track of pending orders and buy price/commission
        self.order = None
        self.buyprice = None
        self.buycomm = None
        self.redline = None
        self.blueline = None

        # Add a BBand indicator
        self.bband = bt.indicators.BBands(self.datas[0], period=self.params.BBandsperiod)

    def notify_trade(self, trade):
        if not trade.isclosed:
            return

        self.log('OPERATION PROFIT, GROSS %.2f, NET %.2f' %
                 (trade.pnl, trade.pnlcomm))

    

    def next(self):
        # Simply log the closing price of the series from the reference
        #self.log('Close, %.2f' % self.dataclose[0])

        # Check if an order is pending ... if yes, we cannot send a 2nd one
        if self.order:
            return

        if self.dataclose < self.bband.lines.bot and not self.position:
        	self.redline = True

        if self.dataclose > self.bband.lines.top and self.position:
        	self.blueline = True

        if self.dataclose > self.bband.lines.mid and not self.position and self.redline:        	
        	# BUY, BUY, BUY!!! (with all possible default parameters)
            self.log('BUY CREATE, %.2f' % self.dataclose[0])
            # Keep track of the created order to avoid a 2nd order
            self.order = self.buy()

        if self.dataclose > self.bband.lines.top and not self.position:
            # BUY, BUY, BUY!!! (with all possible default parameters)
            self.log('BUY CREATE, %.2f' % self.dataclose[0])
            # Keep track of the created order to avoid a 2nd order
            self.order = self.buy()

        if self.dataclose < self.bband.lines.mid and self.position and self.blueline:
            # SELL, SELL, SELL!!! (with all possible default parameters)
            self.log('SELL CREATE, %.2f' % self.dataclose[0])
            self.blueline = False
            self.redline = False
            # Keep track of the created order to avoid a 2nd order
            self.order = self.sell()

In [14]:
def runCerebro(estrategia, results):
    
    for symbol in symbols:
    
        cerebro = bt.Cerebro()
        
        try:
            data = bt.feeds.YahooFinanceData(dataname= symbol+'.SA',
                                         fromdate=datetime(2021, 1, 3),
                                         todate=datetime(2021, 4, 14))

            cerebro.adddata(data)

            cerebro.addstrategy(estrategia['strategy'])

            cerebro.run()

            results.append({
                'symbol': symbol,
                'result': cerebro.broker.getvalue(),
                'estrategia': estrategia['name']
            })
        except Exception:
            pass
        
        
        #plt.rcParams["figure.figsize"] = (15,10)

        #cerebro.plot(iplot=False);
        
    return results

In [15]:
%matplotlib inline

estrategias = ({'name': 'bolinger', 'strategy': TestStrategy},
              {'name': 'sma1030', 'strategy': SmaCross1030},
              {'name': 'sma1020', 'strategy': SmaCross1020})

results = []

for estrategia in estrategias:
    runCerebro(estrategia, results)

        
results

USIM5.SA
2021-02-23, BUY CREATE, 15.74
CIEL3.SA
2021-03-17, BUY CREATE, 3.66
EMBR3.SA
2021-02-18, BUY CREATE, 11.30
BRML3.SA
2021-02-02, BUY CREATE, 9.87
COGN3.SA
2021-03-10, BUY CREATE, 4.05
GRND3.SA
2021-03-05, BUY CREATE, 7.79
ECOR3.SA
2021-03-11, BUY CREATE, 11.57
SMLS3.SA
2021-03-11, BUY CREATE, 19.10
LJQQ3.SA
2021-02-02, BUY CREATE, 19.50
POSI3.SA
2021-03-12, BUY CREATE, 4.82
AZEV4.SA
2021-03-26, BUY CREATE, 3.71
USIM5.SA
2021-02-24 Sinal de compra
2021-04-01 Sinal de venda
USIM5.SA
2021-04-09 Sinal de compra
CIEL3.SA
2021-03-26 Sinal de compra
BRML3.SA
2021-03-22 Sinal de compra
COGN3.SA
2021-03-23 Sinal de compra
2021-04-06 Sinal de venda
COGN3.SA
2021-04-07 Sinal de compra
GRND3.SA
2021-03-16 Sinal de compra
ECOR3.SA
2021-03-24 Sinal de compra
SMLS3.SA
2021-02-22 Sinal de compra
2021-03-02 Sinal de venda
SMLS3.SA
2021-03-18 Sinal de compra
LJQQ3.SA
2021-04-09 Sinal de compra
POSI3.SA
2021-03-18 Sinal de compra
AZEV4.SA
2021-03-31 Sinal de compra
USIM5.SA
2021-02-19 Sinal de co

[{'symbol': 'USIM5', 'result': 10003.390000000001, 'estrategia': 'bolinger'},
 {'symbol': 'CIEL3', 'result': 10000.17, 'estrategia': 'bolinger'},
 {'symbol': 'EMBR3', 'result': 10004.679999999998, 'estrategia': 'bolinger'},
 {'symbol': 'BRML3', 'result': 10000.08, 'estrategia': 'bolinger'},
 {'symbol': 'COGN3', 'result': 10000.01, 'estrategia': 'bolinger'},
 {'symbol': 'GRND3', 'result': 10000.58, 'estrategia': 'bolinger'},
 {'symbol': 'ECOR3', 'result': 10000.33, 'estrategia': 'bolinger'},
 {'symbol': 'SMLS3', 'result': 10002.710000000001, 'estrategia': 'bolinger'},
 {'symbol': 'LJQQ3', 'result': 9997.01, 'estrategia': 'bolinger'},
 {'symbol': 'POSI3', 'result': 10004.06, 'estrategia': 'bolinger'},
 {'symbol': 'AZEV4', 'result': 9999.900000000001, 'estrategia': 'bolinger'},
 {'symbol': 'USIM5', 'result': 9999.6, 'estrategia': 'sma1030'},
 {'symbol': 'CIEL3', 'result': 10000.22, 'estrategia': 'sma1030'},
 {'symbol': 'EMBR3', 'result': 10000.0, 'estrategia': 'sma1030'},
 {'symbol': 'BRM

In [16]:
df = pd.DataFrame(results)

pd.set_option('display.max_rows', df.shape[0]+1)

df.sort_values(by='result', ascending=False).head(100)

,symbol,result,estrategia
2,EMBR3,10004.68,bolinger
24,EMBR3,10004.62,sma1020
9,POSI3,10004.06,bolinger
0,USIM5,10003.39,bolinger
20,POSI3,10003.29,sma1030
31,POSI3,10002.82,sma1020
7,SMLS3,10002.71,bolinger
22,USIM5,10002.47,sma1020
29,SMLS3,10001.08,sma1020
18,SMLS3,10000.88,sma1030


In [17]:
df.sort_values(by='result', ascending=False).to_csv('results_strategies_my.csv', index=False)